In [241]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

In [49]:
def feature_sex(s):
    if(s =='male'):
        return 0
    else:
        return 1

In [169]:
def feature_embarked(s):
    if(s == 'S'):
        return pd.Series([1,0,0])
    elif(s =='C'):
        return pd.Series([0,1,0])
    elif(s =='Q'):
        return pd.Series([0,0,1])
    else:
        return pd.Series([0,0,0])

In [210]:
def convert_df(titanic_df):
    titanic_df[['S','C','Q']] = titanic_df['Embarked'].apply(feature_embarked)
    titanic_df.drop(['Name','Ticket','Cabin','Embarked'],axis=1,inplace=True)
    titanic_df['Age'].fillna(titanic_df['Age'].median(axis=0),inplace=True)
    titanic_df['Sex'] = titanic_df['Sex'].apply(feature_sex)
    return titanic_df

In [354]:
#model = LogisticRegression(C=100)
#model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.05,max_depth=1)
titanic_df = pd.read_csv('train.csv')
pipeline = Pipeline([
        ('standard_scaler', StandardScaler()),
        ('gbdt',GradientBoostingClassifier())])
#params = {
#    'lg__C' : (1, 10,100),
#}
params={"gbdt__n_estimators": list(range(4,20,2)),"gbdt__max_depth": [1,2,4],"gbdt__learning_rate":[0.01,0.05,0.1,0.5]}
clf = GridSearchCV(pipeline, params)
titanic_df = convert_df(titanic_df)
x_train = titanic_df.values[500:,2:]
y_train = titanic_df.values[500:,1:2]
x_test = titanic_df.values[500:,2:]
y_test = titanic_df.values[500:,1:2]
c, r = y_train.shape
y_train = y_train.reshape(c,)

In [355]:
clf.fit(x_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('standard_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('gbdt', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impuri...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'gbdt__n_estimators': [4, 6, 8, 10, 12, 14, 16, 18], 'gbdt__max_depth': [1, 2, 4], 'gbdt__learning_rate': [0.01, 0.05, 0.1, 0.5]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [321]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline, x_train, y_train, scoring='accuracy',cv=3)
#rint("Cross-val accuracy: %f" %scores.mean())
scores

array([ 0.78451178,  0.82828283,  0.8013468 ])

In [356]:
pred = clf.predict(x_test)
clf.best_params_

{'gbdt__learning_rate': 0.05, 'gbdt__max_depth': 4, 'gbdt__n_estimators': 6}

In [351]:
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))
model=clf.best_estimator_
#print(sorted(clf.get_params(True).items()))
#print(clf.coef_)
model

             precision    recall  f1-score   support

        0.0       0.79      0.99      0.88       242
        1.0       0.98      0.57      0.72       149

avg / total       0.86      0.83      0.82       391

[[240   2]
 [ 64  85]]


Pipeline(memory=None,
     steps=[('standard_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('gbdt', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=4,
              max_features=None, max_leaf_nodes=None,
              min_impur...'auto',
              random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])

In [352]:
model = GradientBoostingClassifier(n_estimators=6,max_depth=4,learning_rate=0.05)
x_train = titanic_df.values[:,2:]
y_train = titanic_df.values[:,1:2]
c, r = y_train.shape
y_train = y_train.reshape(c,)
model.fit(x_train,y_train)
test_df = pd.read_csv('test.csv')
test_df = convert_df(test_df)
test_df['Fare'].fillna(0,inplace=True)
test_x = test_df.values[:,1:]
#test_y = test_df.values[:,0:1]
class_predict = model.predict(test_x)

In [353]:
zip_data = zip(test_df.values[:,0].astype(int),class_predict.astype(int))
res = list(zip_data)
res_df = pd.DataFrame(res)
res_df.columns = ["PassengerId","Survived"]
res_df.to_csv("predict_result_data.csv",index=False)

In [128]:
log_model.coef_

array([[-0.95766491,  2.64404305, -0.03403251, -0.32781096, -0.09975941,
         0.00391059]])

In [130]:
log_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [134]:
log_model.predict_proba(test_x)[:

In [138]:
prob[0][0]

0.90838953070541884

In [139]:
titanic_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,0,22.0,1,0,7.2500
1,2,1,1,1,38.0,1,0,71.2833
2,3,1,3,1,26.0,0,0,7.9250
3,4,1,1,1,35.0,1,0,53.1000
4,5,0,3,0,35.0,0,0,8.0500
